In [1]:
import pandas as pd
import numpy as np
import re
import collections, itertools
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.util import ngrams
from nltk.probability import FreqDist
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to C:\Users\Roger
[nltk_data]     Gualberti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Roger
[nltk_data]     Gualberti\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Roger
[nltk_data]     Gualberti\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Roger
[nltk_data]     Gualberti\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
with open('data/text_article_4484494.txt') as f:
    text = f.read()

1. Ponctuation 

In [3]:
text = re.sub('[^A-Za-z0-9]+', ' ', text)

2. Lower all words

In [4]:
text = text.lower()

3. Remove stopwords

In [14]:
stop_words = set(stopwords.words('english'))
word_tokens = word_tokenize(text)
filtered_text = [w for w in word_tokens if not w.lower() in stop_words]

print('Number of words before removing the stop words',len(word_tokens))
print('Number of words after removing the stop words',len(filtered_text))

Number of words before removing the stop words 7863
Number of words after removing the stop words 4042


['operator',
 'welcome',
 'bellring',
 'brands',
 'first',
 'quarter',
 '2022',
 'earnings',
 'conference',
 'call',
 'webcast',
 'hosting',
 'call',
 'today',
 'bellring',
 'brands',
 'darcy',
 'davenport',
 'president',
 'chief',
 'executive',
 'officer',
 'paul',
 'rode',
 'chief',
 'financial',
 'officer',
 'today',
 'call',
 'recorded',
 'available',
 'replay',
 'beginning',
 '1',
 '30',
 'p',
 'eastern',
 'time',
 'dial',
 'number',
 '800',
 '839',
 '8318',
 'pass',
 'code',
 'required',
 'time',
 'participants',
 'placed',
 'listen',
 'mode',
 'pleasure',
 'turn',
 'floor',
 'jennifer',
 'meyer',
 'investor',
 'relations',
 'bellring',
 'brands',
 'introductions',
 'may',
 'begin',
 'jennifer',
 'meyer',
 'good',
 'morning',
 'thank',
 'joining',
 'us',
 'today',
 'bellring',
 'brands',
 'first',
 'quarter',
 'fiscal',
 '22',
 'earnings',
 'call',
 'today',
 'darcy',
 'davenport',
 'president',
 'ceo',
 'paul',
 'rode',
 'cfo',
 'darcy',
 'paul',
 'begin',
 'prepared',
 'remarks

4. Lemmatization

In [6]:
wordnet_lemmatizer = WordNetLemmatizer()

In [7]:
# Lemmatize only the noun
x = [wordnet_lemmatizer.lemmatize(word, pos='n') for word in filtered_text]

5. N-Gram 

In [9]:
def ngrams_list(n):
    """
        Compute ngrams.
        
        Args:
            n (int): the number of words to words to assemble in the ngram.
        
        Returns :
            A list composed of the ngrams.
    """
    m = []
    nx_grams = ngrams(sequence = nltk.word_tokenize(text), n = n)
    for gram in nx_grams:
        m.append(gram)
    return m

6. Remove most frequent and least frequent words

In [10]:
def remove(a,m,l):
    """
        Remove most frequent and least words.
        Args:
            a (list) : list on which operations should be made.
            m (float): the proportion of top l% least frequent words to remove.
            l (float): the proportion of top h% most frequent words to remove.
    
        Returns:
            A copy of the input text text without frequent and infrequent words.
    """
    f = FreqDist(a)
    df_fdist = pd.DataFrame({'Word': f.keys(), 'Frequency': f.values()})
    df_fdesc = df_fdist.sort_values(by='Frequency', ascending=False)
    df_fasc = df_fdist.sort_values(by='Frequency', ascending=True)
    #m = m
    #l = l
    #most_freq_words_list = list(df_fdesc['Word'][0:m])
    #least_freq_word_list = list(df_fasc['Word'][-l:])
    most_freq_words_list = list(df_fdesc[df_fdesc['Frequency'] < m]['Word'])
    least_freq_word_list = list(df_fasc[df_fasc['Frequency'] > l]['Word'])
    text_wo_extremes = ' '.join([word for word in filtered_text if word not in most_freq_words_list or least_freq_word_list]) 
                                                                #how to check on both most_freq and least_freq ?
    return text_wo_extremes

In [11]:
print(remove(filtered_text,0.0001,0.01), '\n-------------------------------------------------', ' '.join(filtered_text))
print(len(remove(filtered_text,10,10)))
#' '.join(filtered_text)
#filtered_text

operator welcome bellring brands first quarter 2022 earnings conference call webcast hosting call today bellring brands darcy davenport president chief executive officer paul rode chief financial officer today call recorded available replay beginning 1 30 p eastern time dial number 800 839 8318 pass code required time participants placed listen mode pleasure turn floor jennifer meyer investor relations bellring brands introductions may begin jennifer meyer good morning thank joining us today bellring brands first quarter fiscal 22 earnings call today darcy davenport president ceo paul rode cfo darcy paul begin prepared remarks afterwards brief question answer session press release supplemental slide presentation support remarks posted web site investor relations sec filings sections bellring com addition release slides available sec web site continue would like remind call contain forward looking statements subject risks uncertainties carefully considered investors actual results could

7. Stemming

In [31]:
def stemming(x):
    stemmer = PorterStemmer()
    stems_list = []
    for w in x:
        stems_list.append(stemmer.stem(w))
    
    df_stems = pd.DataFrame(x, columns = ['Word'])
    df_stems['Stem'] = stems_list
    return df_stems
        
#stemming(filtered_text)